![](img.png)
# LLMs in Real Products
<a target="_blank" href="https://colab.research.google.com/github/life-efficient/OpenAI-Powered-Products/blob/main/Notebook.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Aim: Create an AI powered product to illustrate the general approach to building OpenAI "wrapper startups".

> ### To demonstrate this, we will build: __a product that tells a back and forth story, and creates visualisations of the story as it evolves.__

## Outline

### Part 1: Build an application that runs from a Python notebook
1. Allow the user to type out what happens next in the story
1. Using prompt engineering to pre-process the user's text
1. Make a request to the OpenAI API to continue the story by completing our text
1. Use the moderation API to ensure our responses aren't violating any guidelines
1. Generate images to visualise the story as we progress
1. Run the application in Python as an example

![](./images/Local%20Application.png)

### Part 2: Build a user interface and an API to deploy the application online
1. Wrap all of our code in an API
1. Use ChatGPT to code up a working user interface that could be deployed online
1. Make the user interface interact with our API

![](./images/API%20Application.png)

Let's do this...

## Before we start, it's important to understand what an API is

API stands for Application Programmable Interface
- Simply put, it's a piece of software that can accept "requests" made my code and knows how to respond to different types of requests
  - E.g. the OpenAI API knows how to return a text completion as a response to a request to complete some text
- The OpenAI API is running on a computer in the cloud
- The API we will build in part 2 will be running on your own computer
    - Eventually, you could deploy this to a computer running in the cloud and provide access to everyone on the internet

![](./images/API%20Request%20Response.png)

# Part 1: Build an application that runs from a Python notebook

![](./images/Local%20Application.png)

## Installing dependencies

The cell below runs several commands in the terminal that each install a Python library. If you're on Google Colab, they will install the library to the machine running in the cloud. If you're running this notebook locally, they will install the libraries to your computer.

In [ ]:
# !conda install - y - c conda-forge uvicorn
# !conda install - c conda-forge openai
# !conda install - c conda-forge fastapi


## Setting Up

To set up, you'll need to install the `openai` Python library, and then set your API key. You create/find an API key on your [OpenAI console](https://platform.openai.com/account/api-keys).

I've seen some people have trouble with this, so if you run into issues:
- Ensure you've [created an account on OpenAI](https://platform.openai.com/overview#:~:text=Examples-,Log,-in)
- Ensure you've created an API key [on your account](https://platform.openai.com/account/api-keys)
- Ensure you've replaced my placeholder API key in the next code cell with your own API key
- Ensure you've set up a paid account on OpenAI
    - You will need to add a credit card, but don't worry, the amount of requests we will make will only cost a few pence
- Try [creating a different API key](https://platform.openai.com/account/api-keys) and changing it in the code things still don't work

In [ ]:
import openai

openai.api_key = "YOUR_API_KEY"

## List the available models

The OpenAI API provides many different services including:
- Providing chat responses
- Completing a starting prompt
- Moderating inappropriate text
- Generating images

Under the hood, each of those services is an AI model (a function which takes in some input and makes a prediction about the result, e.g. text -> completion).

Check the comparison in the documentation, [here](https://platform.openai.com/docs/models/gpt-3-5), or uncomment and run the cell below (but prepare for hundreds of lines of output to scroll through).


In [ ]:
# openai.Model.list() # shows hundreds of lines

## Extra: Fine tune a model

If you acquire your own dataset, you can fine-tune a model to your specific domain.

Check out the details [here](https://platform.openai.com/docs/guides/fine-tuning).

## Requesting Text Completions

Firstly, let's get a hang of how to use one of the chat-optimised models, `gpt-3.5-turbo`.

As I always emphasise, the real skill is being able to read the documentation. Check it out [here](https://platform.openai.com/docs/api-reference/completions).

Make sure to take a look into the different parameters and what they do.

In [ ]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a story!"}
    ]
)

print(completion)


## Unpacking the response

The response that you get back contains more than just the text completion. As a next step, write a line of code to get just the text out.

In [ ]:
prediction = completion.choices[0].message["content"]
print(prediction)

### Wrap all of this into a function

In our case, both making the request and unpacking the response are things that are always going to need to happen together, so we should encapsulate them into the same function.

You'll need to:
- Pass in a list representing the sequence of messages which have been said in the chat so far, rather than just a single prompt

In [ ]:
def get_completion(messages):

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    prediction = completion.choices[0].message["content"]
    return prediction


print("EXAMPLE 1")
messages = [
    {"role": "user", "content": "What is the meaning of life?"}
]
print(get_completion(messages))


print("EXAMPLE 2")
messages = [
    {"role": "user", "content": "Hi, how are you?"},
    {"role": "assistant", "content": "Hi, how are you?"},
    {"role": "user", "content": "I'm good. What's for lunch?"}
]
print(get_completion(messages))

## Moderations

As a precaution against innapropriate messaging getting out to your users, you can run responses from the models through the moderations API.

It's unlikely that the OpenAI models will respond with anything inappropriate, because they're aligned not to. It's more likely that this should be used to make sure your user requests are not harmful.

In [ ]:
response = get_completion([{"role": "user", "content": "Tell me something inappropriate."}])

print(response)

result = openai.Moderation.create(input=response)
print(result)

## Now let's combine the code into a prototype application

You will need to:
- Prompt the system to provide a start to the story
- Then in a loop:
    - Ask the user to input what happens next
    - Check that the user input does not violate the content policy
    - Ask OpenAI to continue the story

Feel free to add extra functionality!

> Note: You'll need to "interrupt" the code cell by hitting the stop icon to run the code again after you make changes, so that it doesn't get stuck in the infinite loop.

In [ ]:
def get_next_line(messages):
    response = get_completion(messages)
    print("AI:", response)
    messages.append({"role": "assistant", "content": response})

    message = input("You: ")
    print("You:", message)
    messages.append({"role": "user", "content": message})
    messages.append({"role": "user", "content": "Tell me the next line of the story!"})
    return messages


def main():
    messages = [{"role": "user", "content": "Give me the first line of a story!"}]
    while True:
        messages = get_next_line(messages)


main()


## Adding AI Generated Images

To immerse our users in the story, let's use the OpenAI [image generation API endpoint](https://platform.openai.com/docs/api-reference/images) to visualise what the most recent scene in the story looks like.

This can be accomplished using OpenAI's image generation endpoint, which creates an image from a text prompt.

To do this, we'll:
1. Take part of the story so far and transform it into an image prompt
1. Make a request to generate an image from that prompt
1. Unpack the image response
1. Show the image to the user (or just give them the link to open for now - it's hard to visualise images in a notebook)

## Generating the image prompt

In [ ]:
def create_image_prompt_from_text(text):
    print("TEXT TO IMAGE PROMPT", text)

    text = """
    Distil the following text into a rich visual description of a picture from the scene in less than 400 characters.

    Describe the subject and surrounding scene first. Then go on to describe the angle and field of view of the camera, the style of the illustration, the lighting, and the mood.

    Story:
    """ + text

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": text}
        ]
    )
    response = response.choices[0].message["content"]
    return response

story = get_completion([{"role": "user", "content": "Give me the first line of a story!"}])
print("STORY")
print(story)
print("\n\n\n")
scene = create_image_prompt_from_text(story)
print("SCENE")
print(scene)


## Generate an Image from the Prompt

Now you've got the description of an image that visualises this part of the story, you can generate an image from it using DALL-E, OpenAI's image generation API.

In [ ]:
def generate_image_from_prompt(prompt):
    response = openai.Image.create(
        prompt=prompt,
        n=2,
        size="256x256"
    )
    return response


image = generate_image_from_prompt(scene)
print(image)



### Unpacking the image from the response

Again, the response contains more than just the image, so we'll need to extract the image URL.

In [ ]:
image_url = image["data"][0]["url"]


## The following cell displays an image in a notebook

In [ ]:
from IPython.display import Image

Image(url=image_url) 

In [ ]:
def create_and_display_image_from_prompt(prompt):
    print(prompt)
    image = generate_image_from_prompt(prompt)
    image_url = image["data"][0]["url"]
    print(image_url)
    return Image(url=image_url)

create_and_display_image_from_prompt(scene)

### Tying that all together into our application

Let's take everything we've built so far and run that as a single application

In [ ]:
def create_story_with_images():
    messages = [{"role": "user", "content": "Give me the first line of a story!"}]
    while True:
        messages = get_next_line(messages)
        print(messages)
        last_scene = messages[-2]["content"]
        image_prompt = create_image_prompt_from_text(last_scene)
        create_and_display_image_from_prompt(image_prompt)

create_story_with_images()


# Part 2: Build a user interface and an API to deploy the application online

To serve our application as a product, we need to put all of this in a Python file, then turn it into an API

![](./images/API%20Application.png)

## Move to VSCode

VSCode is the industry standard code development environment - a place for writing code.

I know one of you is about to ask whether they can use a different editor like Spyder or PyCharm, but please just get with the program and use VSCode - it's professional and almost certainly what you'd be required to use in industry. 
We will use a VSCode extension to run the user interface that shows the application.
If you refuse to use VSCode you'll need to find another way of running the user interface, and I'm not sure how to do that.

> ### At this point, you should move away from Google Colab, and instead to VSCode then start working in Python files (`.py`)

In VSCode, put the utilities you've developed in this file into a Python file called `utils.py`.

## Building the back-end

The "back-end" of our application will be an API (application programmable interface). Here's the essentials of what you need to know about an API:
- Simply a program which runs on a computer somewhere, typically in a datacentre (on the cloud)
- Can accept "requests" and return responses
- Can implement different, custom responses to different requests

Our API will be able to respond to requests for things like:
- Text completion
- Moderation
- Converting our story so far into an image prompt
- Getting recommendations for follow-up prompts

_FastAPI_ is a Python library that can be used to create APIs. It defines simple ways to define the different things that your API can respond to. Read more about FastAPI [here](https://fastapi.tiangolo.com/).

Here's the basic structure:

In [ ]:
# highlight the basic strcuture for using fastapi

import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class EndpointData(BaseModel): # define the schema for the incoming data
    text: str # this schema expects an attribute of type string called "text"

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.post("/demo-endpoint") # define what happens to POST requests made to the /chat-response endpoint 
def predict(endpoint_data: EndpointData): # define a method that expects the incoming data to follow the Chat schema
    print("incoming payload text attribute:", endpoint_data.text)
    return {"response": "This is a response"} # return a response

if __name__ == '__main__': # if this file is being run directly
    uvicorn.run(app, host='localhost', port=8000) # run the API

# run the following from terminal to run this API: `uvicorn get_prediction:app --reload`

Before this works, you'll need to use `conda` to install some code that it depends on. You may want to do this in a new and separate conda environment.

In [ ]:
!conda install -y -c conda-forge uvicorn
!conda install -c conda-forge openai
!conda install -c conda-forge fastapi


Note again, this API code should be in a different `.py` file. To run it, run `uvicorn get_prediction: app --reload` from the terminal.

The API will continue to run until you kill the terminal process or until it runs into an error.

Remember that the API needs to be running still.

## Testing the back-end

We can test the back end by running it locally, and then making requests to it via Python.

There are other ways to make these request too, including:
- Using `curl` from the command line
- Opening the [URL](http://localhost:8000/demo-endpoint) that's serving the webpage


In [ ]:
import requests

response = requests.post("http://localhost:8000/demo-endpoint", json={"text": "This is a test"})
print(response)
print(response.json())

Now that the back end is working, you can begin to connect the front end to it. I've included a file called `index.html` that contains HTML which defines the structure of the webpage.

To run it locally with minimal hassle, click on the extensions tab on the far left of VSCode and download the extension shown in the following image.

![](./images/extension.png)

As per the instructions shown, you can hit `CTRL + SHIFT + L` when you're inside an HTML file, and it will run a preview of that webpage on a tab in your browser. It also updates as soon as you hit save.

When you run that you should see something like this:

![](./images/app.png)

If you take a look into the HTML file you can begin to decipher how the code works. You'll see it contains not just HTML, but also CSS and JavaScript.

But there's one secret weapon you can use here even if you don't know HTML, CSS, JavaScript... You can ask ChatGPT to write the code for you.

## Challenge: Use what you've learnt so far to build your own application

You should:
- Ask ChatGPT to help you write the UI
    - I don't expect you to know HTML, JS, or CSS, and you won't be responsible for this in industry either
    - If you're in lecture with me, then I can help you write this


## Key Takeaways
- The OpenAI API is extremely easy to use
- Many products can be built using prompt engineering to adjust the input to OpenAI API endpoints to do things like create images and respond to a chat
- You can easily build a simple prototype of a product with just a few tools, in just a few hours
- You can use ChatGPT to write a mock up of the front-end with you